<a href="https://colab.research.google.com/github/danielbehargithub/LinkedIn_Salary/blob/main/Wage_and_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install all dependencies
!pip install gensim
!pip install nltk
!pip install pandas
!pip install numpy
!pip install requests
!pip install playwright
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 12.7 MB/s eta 0:00:00
161.3 MiB [] 0% 0.0s161.3 MiB [] 0% 65.7s161.3 MiB [] 0% 31.0s161.3 MiB [] 0% 20.6s161.3 MiB [] 0% 14.3s161.3 MiB [] 0% 12.9s161.3 MiB [] 0% 12.7s161.3 MiB [] 1% 12.4s161.3 MiB [] 1% 11.8s161.3 MiB [] 1% 11.3s161.3 MiB [] 1% 10.2s161.3 MiB [] 2% 10.0s161.3 MiB [] 2% 9.5s161.3 MiB [] 2% 9.4s161.3 MiB [] 2% 9.0s161.3 MiB [] 3% 8.8s161.3 MiB [] 3% 8.5s161.3 MiB [] 3% 8.0s161.3 MiB [] 4% 7.6s161.3 MiB [] 4% 7.4s161.3 MiB [] 4% 7.1s161.3 MiB [] 5% 7.0s161.3 MiB [] 5% 6.8s161.3 MiB [] 5% 6.7s161.3 MiB [] 6% 6.5s161.3 MiB [] 6% 6.7s161.3 MiB [] 6% 6.6s161.3 MiB [] 6% 7.0s161.3 MiB [] 6% 7.4s161.3 MiB [] 6% 7.6s161.3 MiB [] 7% 7.6s161.3 MiB [] 7% 8.0s161.3 MiB [] 7% 8.1s161.3 MiB [] 7% 8.3s161.3 MiB [] 7% 8.4s161.3 MiB [] 7% 8.3s161.3 MiB [] 8% 8.3s161.3 MiB [] 8% 8.4s161.3 MiB [] 8% 8.5s161.3 MiB [] 8% 8.4s161.3 MiB [] 9% 8.0s161.3 MiB [] 9% 7.8s161.3 MiB [] 10% 7.6s161.3 MiB [] 10% 7.4s161.3 MiB [] 10% 7.3s161.3 M

In [ ]:
# Import libraries
from gensim.models.doc2vec import Doc2Vec, TaggedDocument,Word2Vec
from nltk.tokenize import word_tokenize
from numpy.linalg import norm
from termcolor import colored
import pandas as pd
import numpy as np
import requests
import re
import plotly.graph_objects as go
import nltk
nltk.download('all', download_dir='/usr/local/lib/nltk_data')
from nltk import pos_tag
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer, util

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package avera

In [ ]:
job_details_file = "job_details.txt"
profile_data_file = "profile_data.txt"


In [ ]:
def extract_section(file_path, start_keyword, end_keyword):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    extracting = False
    extracted_text = []

    for line in lines:
        if start_keyword in line:
            extracting = True
        if extracting:
            extracted_text.append(line.strip())
        if end_keyword in line:
            break

    return "\n".join(extracted_text)

unique_skills_raw = extract_section(profile_data_file, "Unique skills:", "Unique skills:")

if unique_skills_raw.startswith("Unique skills:"):
    unique_skills = unique_skills_raw.replace("Unique skills:", "").strip()

# Convert skills to a formatted string
profile_skills = f"""['{unique_skills.replace(', ', "', '")}']"""



In [ ]:
def extract_description(file_path, start_keyword, end_keyword=None):
    """
    Extract a specific section of text from a file based on start and optional end keywords.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    extracting = False
    extracted_text = []

    for line in lines:
        if start_keyword in line:
            extracting = True
        if extracting:
            extracted_text.append(line.rstrip())  # Keep formatting with spaces
        if end_keyword and end_keyword in line:
            break

    return "\n".join(extracted_text)

# File path to the uploaded text file
file_path = "job_details.txt"

# Extract the description section
job_description = extract_description(file_path, "Job description", "Description:")



In [ ]:
# Extended stopwords for job descriptions
stop_words = set(stopwords.words('english')).union({
    'have', 'has', 'you', 'we', 'they', 'our', 'your', 'the', 'a', 'an', 'in', 'on', 'at', 'for', 'with', 'to', 'of',
    'and', 'or', 'is', 'are', 'was', 'were', 'be', 'will', 'can', 'should', 'would', 'could', 'may', 'might',
    'solid', 'relevant', 'extensive', 'proven', 'experience', 'possess', 'ability', 'skills', 'capabilities',
    'demonstrated', 'strong', 'knowledge', 'understanding', 'background', 'proficiency', 'required',
    'responsibilities', 'qualifications', 'advantage', 'must', 'preferred', 'environment', 'excellent', 'good',
    'key', 'various', 'well', 'including', 'new', 'opportunities', 'job', 'role', 'position'
})

# 1. Preprocessing Functions
def preprocess_text(text):
    """
    Preprocess text by removing excessive newlines, extra spaces, and special characters.
    """
    if not isinstance(text, str) or not text.strip():
        return ""
    text = re.sub(r'\s*\n\s*', ' ', text)  # Replace newlines with spaces
    text = re.sub(r'\s+', ' ', text)       # Replace multiple spaces with a single space
    text = re.sub(r'[^\w\s]', '', text)    # Remove special characters
    return text.strip().lower()            # Lowercase and strip whitespace

def extract_relevant_phrases(sentence):
    """
    Extract relevant phrases from a sentence by removing unnecessary words and focusing on key phrases.
    """
    # Remove unnecessary phrases using regex
    cleaned_sentence = re.sub(r'\b(have|has|possess|proven|solid|relevant|extensive|demonstrated|strong)\b', '', sentence, flags=re.IGNORECASE)
    cleaned_sentence = re.sub(r'\b(experience (in|with|on|of|for))\b', '', cleaned_sentence, flags=re.IGNORECASE)
    cleaned_sentence = re.sub(r'\b(ability to|proficiency in|knowledge of|understanding of|background in)\b', '', cleaned_sentence, flags=re.IGNORECASE)

    # Tokenize and remove stopwords
    tokens = word_tokenize(cleaned_sentence.lower())
    tokens = [token for token in tokens if token not in stop_words and token.isalpha()]

    # Extract nouns using POS tagging
    pos_tags = pos_tag(tokens)
    relevant_phrases = [word for word, tag in pos_tags if tag.startswith('NN')]  # Keep only nouns

    return ' '.join(relevant_phrases)

def process_job_description(description):
    """
    Process job description to split into sentences and extract relevant parts.
    """
    # Split the description into sentences using punctuation or bullet points
    sentences = re.split(r'[.\n•;]', description)
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]  # Remove empty strings

    # Further split sentences into manageable phrases if too long
    processed_sentences = []
    for sentence in sentences:
        # Split sentences longer than a threshold into smaller parts by words
        if len(sentence.split()) > 20:  # Threshold: 20 words
            chunks = re.split(r'(?:,| and | or )', sentence)  # Split by commas, "and", or "or"
            processed_sentences.extend([chunk.strip() for chunk in chunks if chunk.strip()])
        else:
            processed_sentences.append(sentence)

    # Extract relevant phrases from each chunk
    processed_chunks = [extract_relevant_phrases(preprocess_text(chunk)) for chunk in processed_sentences]

    # Remove empty results
    return [chunk for chunk in processed_chunks if chunk]

# 2. Main Workflow
def compute_similarity(job_description, profile_skills, model, threshold=0.5):
    """
    Compute similarity between job description and profile skills using embeddings.
    """
    # Preprocess inputs
    job_skills_processed = process_job_description(preprocess_text(job_description))
    profile_skills_list = [preprocess_text(skill) for skill in eval(profile_skills)]  # Convert and preprocess profile skills

    # Encode skills using Sentence-BERT
    job_embeddings = model.encode(job_skills_processed, convert_to_tensor=True)
    profile_embeddings = model.encode(profile_skills_list, convert_to_tensor=True)

    # Compute cosine similarities
    cosine_similarities = util.pytorch_cos_sim(job_embeddings, profile_embeddings)

    # Adjust similarities based on threshold
    adjusted_similarities = []
    total_similarity = 0
    comparison_count = 0

    for i in range(len(job_skills_processed)):
        similarity_row = cosine_similarities[i].tolist()
        max_similarity = max(similarity_row)
        if max_similarity > threshold:
            similarity_row = [1.0] * len(similarity_row)
        adjusted_similarities.append(similarity_row)
        total_similarity += sum(similarity_row)
        comparison_count += len(similarity_row)

    regulezation_factor = 0.3
    # Calculate overall similarity
    overall_similarity = min(total_similarity / comparison_count + regulezation_factor, 1.0) if comparison_count > 0 else 0

    return adjusted_similarities, overall_similarity, job_skills_processed, profile_skills_list


In [ ]:

model = SentenceTransformer('all-MiniLM-L6-v2')



# Compute similarities
adjusted_similarities, overall_similarity, job_skills_processed, profile_skills_processed = compute_similarity(
    job_description, profile_skills, model
)

# Display results
print("Adjusted Similarities:")
for i, job_skill in enumerate(job_skills_processed):
    for j, profile_skill in enumerate(profile_skills_processed):
        similarity_score = adjusted_similarities[i][j]
        print(f"Similarity between job skill '{job_skill}' and profile skill '{profile_skill}': {similarity_score:.2f}")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Adjusted Similarities:
Similarity between job skill 'description algorithm group' and profile skill 'analytical skills': 0.14
Similarity between job skill 'description algorithm group' and profile skill 'candidate management': 0.16
Similarity between job skill 'description algorithm group' and profile skill 'cosourcing': 0.11
Similarity between job skill 'description algorithm group' and profile skill 'communication': 0.23
Similarity between job skill 'description algorithm group' and profile skill 'corporate recruiting': 0.17
Similarity between job skill 'description algorithm group' and profile skill 'curriculum vitae cv': -0.02
Similarity between job skill 'description algorithm group' and profile skill 'english': 0.10
Similarity between job skill 'description algorithm group' and profile skill 'executive search': 0.18
Similarity between job skill 'description algorithm group' and profile skill 'game theory': 0.12
Similarity between job skill 'description algorithm group' and profil

In [ ]:
print(f"\nOverall Similarity: {overall_similarity:.2f}")


Overall Similarity: 0.62


In [ ]:
# Assume similarity is your final score as a percentage
similarity = overall_similarity * 100  # Convert to percentage if in range [0, 1]


In [ ]:

# Visualization
fig = go.Figure(go.Indicator(
    domain={'x': [0, 1], 'y': [0, 1]},
    value=similarity,
    mode="gauge+number",
    title={'text': "Matching percentage (%)"},
    gauge={
        'axis': {'range': [0, 100]},
        'steps': [
            {'range': [0, 50], 'color': "#FFB6C1"},  # Light pink for low match
            {'range': [50, 70], 'color': "#FFFFE0"},  # Light yellow for moderate match
            {'range': [70, 100], 'color': "#90EE90"}  # Light green for high match
        ],
        'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': similarity}  # Mark the current similarity
    }
))

fig.update_layout(width=600, height=400)  # Adjust the width and height as desired
fig.show()


In [ ]:
!pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 27.6 MB/s eta 0:00:00


In [ ]:
def categorize_experience(profile_text):
    """
    פונקציה שממפה קטגוריית ניסיון (EN, MI, SE, EX) על סמך שנות ניסיון מתוכן הטקסט.
    :param profile_text: טקסט פרופיל משתמש (string)
    :return: קטגוריית ניסיון (string)
    """
    import re

    # חיפוש שנות ניסיון בטקסט (למשל: '2 yrs', '10 years', '1 yr')
    experience_matches = re.findall(r"(\d+)\s*(?:yrs?|years?)", profile_text)

    # המרה למספרים של שנות ניסיון
    years_of_experience = sum(map(int, experience_matches))

    # קטגוריות ניסיון לפי מספר שנים
    if years_of_experience < 2:
        return 'EN'  # Entry Level
    elif 2 <= years_of_experience < 5:
        return 'MI'  # Mid Level
    elif 5 <= years_of_experience < 10:
        return 'SE'  # Senior Level
    else:
        return 'EX'  # Executive Level


# הפעלת הפונקציה
experience_category = categorize_experience(profile_data_file)


Experience Category: EN


In [ ]:
import pycountry
import re

def extract_job_details_with_iso(job_text):
    """
    חלץ פרטי משרה מתוך טקסט נתון והמר מיקום החברה ל-ISO 3166 Country Code באמצעות pycountry
    :param job_text: טקסט עם פרטי משרה
    :return: מילון עם פרטי המשרה, כולל ISO Country Code
    """

    # חילוץ כותרת המשרה
    job_title = re.search(r"Job Title\s*:\s*(.+)", job_text)
    job_title = job_title.group(1).strip() if job_title else "N/A"

    # חילוץ שם החברה
    company_name = re.search(r"Company\s*:\s*(.+)", job_text)
    company_name = company_name.group(1).strip() if company_name else "N/A"

    # חילוץ מיקום החברה
    company_location = re.search(r"Location\s*:\s*(.+)", job_text)
    company_location = company_location.group(1).strip() if company_location else "N/A"

    # המרת מיקום החברה ל-ISO 3166 Country Code
    country_code = "N/A"
    for country in pycountry.countries:
        if country.name in company_location:
            country_code = country.alpha_2
            break

    # עבודה מהבית: הגדרה לפי תנאים
    if "Remote" in job_text:
        remote_ratio = 100
    elif "Hybrid" in job_text:
        remote_ratio = 50
    elif "On-site" in job_text:
        remote_ratio = 0
    else:
        remote_ratio = "N/A"

    # הרכבת התוצאה
    return {
        "job_title": job_title,
        "company_name": company_name,
        "company_location": company_location,
        "iso_country_code": country_code,
        "remote_ratio": remote_ratio,
        "company_size": "N/A"  # יכול להיות מוגדר על פי מידע נוסף
    }



# הפעלת הפונקציה
job_details = extract_job_details_with_iso(job_details_file)



{'job_title': 'N/A', 'company_name': 'N/A', 'company_location': 'N/A', 'iso_country_code': 'N/A', 'remote_ratio': 'N/A', 'company_size': 'N/A'}


In [ ]:
import lightgbm as lgb
model = lgb.Booster(model_file='lightgbm_model.txt')
import pandas as pd

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning:


Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.




In [ ]:
new_user = pd.DataFrame({
    'experience_level': [experience_category],         # Senior level
    'company_location': [job_details["iso_country_code"]],         # Company located in the US
    'job_title': [job_details["job_title"]],    # Job title
})



# עמודות קטגוריאליות
categorical_columns = [
    'experience_level', 'employment_type', 'employee_residence',
    'company_location', 'company_size', 'job_title', 'remote_ratio'
]

# המרת עמודות לקטגוריה
for col in categorical_columns:
    if col in new_user.columns:
        new_user[col] = new_user[col].astype('category')

# ווידוא עמודות רלוונטיות לפי X_train
for col in categorical_columns:
    if col not in new_user.columns:
        # הוספת עמודה קטגוריאלית עם ערך ברירת מחדל 'Unknown'
        new_user[col] = "Unknown"
        new_user[col] = new_user[col].astype('category')

# חיזוי שכר
predicted_salary = model.predict(new_user)[0]


In [ ]:
def calculate_salary_range_dynamic(predicted_salary, match_percentage, base_lower_pct=0.9, base_upper_pct=1.0, step_pct=0.05):
    """
    חישוב טווח שכר דינמי המבוסס על רמות שונות של אחוז התאמה, עם ערכים נגזרים באחוזים מהשכר החזוי
    :param predicted_salary: שכר חזוי
    :param match_percentage: אחוז התאמה למשרה
    :param base_lower_pct: אחוז לתחתית הטווח הבסיסי (ברירת מחדל: 90% מהשכר החזוי)
    :param base_upper_pct: אחוז לחלק העליון של הטווח הבסיסי (ברירת מחדל: 100% מהשכר החזוי)
    :param step_pct: אחוז קפיצה לכל רמת התאמה (ברירת מחדל: 5%)
    :return: גבול תחתון ועליון לטווח השכר
    """
    # חישוב טווח בסיסי
    base_lower = predicted_salary * base_lower_pct
    base_upper = predicted_salary * base_upper_pct

    # חישוב גודל הקפיצה באחוזים
    step_size = predicted_salary * step_pct

    # קביעת מספר הצעדים מאחוז התאמה בסיסי (50%)
    steps = (match_percentage - 50) // 10  # כל 10% מייצגים צעד אחד

    # חישוב הטווח החדש
    lower_bound = base_lower + steps * step_size
    upper_bound = base_upper + steps * step_size

    return lower_bound, upper_bound


# דוגמה לשימוש
match_percentage = similarity       # אחוז התאמה לדוגמה

# חישוב טווח השכר
lower_bound, upper_bound = calculate_salary_range_dynamic(predicted_salary, match_percentage)

# הצגת התוצאה
print(f"Predicted Salary: ${predicted_salary:,.2f}")
print(f"Salary Range (Match Percentage = {match_percentage}%): ${lower_bound:,.2f} - ${upper_bound:,.2f}")


Predicted Salary: $55,800.00
Salary Range (Match Percentage = 61.9%): $53,010.00 - $58,590.00


In [ ]:
import plotly.graph_objects as go

# יצירת גרף
fig = go.Figure()

# הוספת טווח השכר כקו ירוק
fig.add_trace(go.Scatter(
    x=[lower_bound, upper_bound],
    y=[0, 0],
    mode='lines+markers',
    line=dict(color='green', width=8, dash='solid'),  # קו ירוק ברור
    marker=dict(size=14, color='green'),
    name="Salary Range"
))

# הוספת השכר החזוי כנקודה אדומה
fig.add_trace(go.Scatter(
    x=[predicted_salary],
    y=[0],
    mode='markers+text',
    text=[f"${predicted_salary / 1000:.1f}K"],  # ערך החיזוי בטקסט
    textposition="top center",
    marker=dict(color='red', size=12),
    name="Predicted Salary"
))

# עיצוב ציר ה-X
fig.update_xaxes(
    tickformat="$,",
    showgrid=True,
    range=[lower_bound - 5000, upper_bound + 5000],  # הרחבת הטווח מסביב לטווח השכר
    tickvals=[lower_bound - 5000, lower_bound, predicted_salary, upper_bound, upper_bound + 5000],
    ticktext=[f"${(lower_bound - 5000) / 1000:.1f}K", f"${lower_bound / 1000:.1f}K",
              f"${predicted_salary / 1000:.1f}K", f"${upper_bound / 1000:.1f}K",
              f"${(upper_bound + 5000) / 1000:.1f}K"]
)

# עיצוב כללי
# עיצוב הגרף
fig.update_layout(
    title="Salary Range",
    xaxis_title="Annual Salary ($)",
    yaxis=dict(visible=False),  # הסתרת ציר Y
    xaxis=dict(showgrid=False, range=[lower_bound - 5000, upper_bound + 5000]),
    showlegend=False,
    width=800,
    height=200
)
fig.show()
